In [1]:
import sys
import os

# Adiciona o diretório raiz do projeto ao sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [2]:
from src.backend.consumer import Consumer
from src.backend.report import Report

In [16]:
from tinydb import TinyDB
from pandas import DataFrame

db = TinyDB(r'C:\Users\user\Desktop\UFPB\5-Periodo\ML\Projeto\consumidor_gov\src\backend\database\consumidor_gov_brasil.db')
reports = db.table('reports')
predictions = db.table('predicitons')
companies = db.table('companies')
users = db.table('users')

In [23]:
for table in db.tables():
    print('Tabela: ', table)
    print('Colunas: ', db.table(table).all()[0].keys())
    print('')


Tabela:  users
Colunas:  dict_keys(['id_user', 'name', 'user_type', 'id_company', 'email', 'cpf_user', 'password'])

Tabela:  reports
Colunas:  dict_keys(['status', 'date', 'report', 'company_response', 'response_date', 'rating_score', 'consumer_written_evaluation', 'id_company', 'id_user', 'id_report'])

Tabela:  companies
Colunas:  dict_keys(['id_company', 'company_name'])

Tabela:  predicitons
Colunas:  dict_keys(['id_report', 'prediction', 'predicted_label', 'true_label'])



In [17]:
from tinydb import Query
query = Query()

my_user = users.search(query.name == 'Usuário de Teste')

In [18]:
my_user

[{'id_user': 2000,
  'name': 'Usuário de Teste',
  'user_type': 'cidadao',
  'id_company': None,
  'cpf_user': '99999999999',
  'email': 'usuario-teste@gmail.com',
  'password': '$2b$12$b8H8yrsok03aPsCy9IOGaumKjaDTwqF0HrC6qaSiFzOZfLPhnDa3K'}]

In [13]:
df_reports = DataFrame(reports)
df_predictions = DataFrame(predictions)
df_companies = DataFrame(companies)

In [14]:
df = (
    df_reports
    .merge(df_predictions, on='id_report', how='left')
    .merge(df_companies, on='id_company', how='left')
    .assign(non_resolution_prob=lambda x: 1 - x.prediction)
    .sort_values('prediction', ascending=False)
)

In [32]:
from pandas import DataFrame
reports_df = DataFrame(reports.all())


In [33]:
reports_df['status'].value_counts()

status
Não avaliado pelo consumidor    18325
Resolvido                       16045
Não Resolvido                   15634
Name: count, dtype: int64

In [31]:
from pandas import to_datetime
from numpy import nan

reports_df.assign(
    date_format=lambda x: to_datetime(x.date.str.slice(0, 10), dayfirst=True),
    rating = lambda x: x.rating_score.str.extract('(\d+)').astype('Int64')
 
).query('rating != @nan').assign(
    rating = lambda x: x.rating.astype(int)
).info()


<class 'pandas.core.frame.DataFrame'>
Index: 31675 entries, 0 to 49989
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          31675 non-null  object        
 1   rating_score  31675 non-null  object        
 2   date_format   31675 non-null  datetime64[ns]
 3   rating        31675 non-null  int32         
dtypes: datetime64[ns](1), int32(1), object(2)
memory usage: 1.1+ MB


In [6]:
from tinydb import Query
reports.search(Query().id_company == 736)

[{'id_company': 736, 'company_name': 'Serasa Experian'}]

In [33]:
sample

{'id_report': 1,
 'prediction': 0.22852706909179688,
 'predicted_label': 0,
 'true_label': 0}

In [21]:
report = Report(
    status=sample["status"],
    date=sample["date"],
    report=sample["report"],
    company_response=sample["company_response"],
    response_date=sample["response_date"],
    rating_score=sample["rating_score"],
    consumer_written_evaluation=sample["consumer_written_evaluation"],
    id_company=sample["id_company"],
    id_user=sample["id_user"],
    id_report=sample["id_report"],
    
)


In [22]:
sample

{'status': 'Não Resolvido',
 'date': '25/04/2025, Santa Maria - RS',
 'report': 'Solicito a imediata retirada da negativação indevida referente a dívida prescrita, com mais de 5 anos, registrada de forma ilegal no Serasa. Caso não seja retirada, tomarei as medidas judiciais cabíveis, inclusive pedido de indenização por danos morais.',
 'company_response': 'Oi, FLAVIA. Tudo bem?\n\n\nAcabamos de responder a sua solicitação. Por favor, acesse o campo "Complemento da Reclamação" e confira a resposta.\n\n\nConte com o nosso apoio.\n\n\nEquipe Serasa Experian.',
 'response_date': '(no mesmo dia)',
 'rating_score': 'Nota 2',
 'consumer_written_evaluation': '<não há comentários do consumidor>',
 'id_company': 736,
 'id_user': 2211,
 'id_report': 1}

In [23]:
consumer = ConsumerPage(None)    

In [24]:
predictions = consumer.predict_report(report)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step


In [25]:
predictions

{'report_id': 1,
 'probability': array([[0.22852713]], dtype=float32),
 'label': array([0], dtype=int64)}